In [ ]:
import xml.etree.ElementTree as Et

from pyserini.search import pysearch
from pyserini.search import pyquerybuilder
from tqdm import tqdm
from collections import defaultdict
from pprint import pprint

import subprocess

from pyserini.pyclass import autoclass
from pyserini.analysis.pyanalysis import get_lucene_analyzer

In [ ]:
run_no = 1 # 1 or 2

In [ ]:
tree = Et.parse('topics-rnd1.xml')
root = tree.getroot()

In [8]:
searcher = pysearch.SimpleSearcher('/home/chris/data/anserini/lucene-index-covid-2020-04-10/')

In [9]:
tasks = [
    ['environmental transmission', 'incubation', 'contagious', 'persistence', 'stability', 'physical', 'weather',
     'epidemiology', 'shedding', 'reproductive number', 'modes of transmission', 'virulent', 'asymptomatic', 'pathogen',
     'evolutionary host', 'transmission host'],
    ['smoking', 'risk', 'pulmonary', 'pre-condition', 'co-infection', 'high-risk', 'severe', 'susceptible', 'fatality',
     'neonates', 'respitory', 'condition', 'pre-existing', 'pregnant', 'morbidities'],
    ['human-animal', 'origin', 'genetics', 'evolution', 'genome', 'sample sets', 'genomic', 'strain', 'livestock',
     'animal host', 'natural history', 'genetic drift', 'mutation', 'genomics', 'sequencing'],
    ['vaccine', 'therapeutic', 'treat', 'drugs', 'pharmaceuticals', 'recipients', 'ADE', 'complication', 'antiviral',
     'prophylaxis', 'cloroquine', 'vaccination', 'immume respone'],
    ['medical care', 'surge capacity', 'nursing home', 'allocation', 'personal protective equirement',
     'clinical characterization', 'nursing', 'care', 'Extracorporeal membrane oxygenation', 'ECMO',
     'mechanical ventilation', 'extrapulmonary manifestations', 'cardiomyopathy', 'cardiac arrest',
     'regulatory standards', 'N95 masks', 'elastomeric respirators', 'telemedicine', 'steroids', 'high flow oxygen',
     'supportive interventions'],
    ['NPI', 'non-pharmaceutical intervention', 'school closure', 'travel ban', 'quarantine', 'mass gathering',
     'social distancing', 'public health advice', 'economic impact'],
    ['counties', 'geographic', 'geography', 'mortality rate', 'spread', 'mutations'],
    ['diagnostic', 'surveillance', 'detection', 'screening', 'ELISAs', 'capacity', 'testing', 'point-of-care',
     'rapid testing', 'pathogen', 'reagent', 'cytokines', 'response markers', 'swabs'],
    ['ethical', 'social science', 'principles', 'standards', 'ethics', 'psychological health', 'fear', 'anxiety',
     'stigma', 'sociology'],
    ['collaboration', 'nomenclature', 'data standards', 'information sharing', 'communication', 'collaborate',
     'coordination', 'misunderstanding', 'action plan']
]

In [ ]:
round_1_topic_task_manual = [2, 0, 3, 0, 3, 7, 3, 7, 6, 5, 4, 5, 0, 0, 0, 0, 7, 5, 5, 1, 0, 0, 1, 1, 1, 7, 7, 3, 3, 3]
round_1_topic_task_automatic = [2, 5, 3, 0, 3, 6, 3, 3, 3, 5, 4, 3, 7, 3, 3, 0, 3, 0, 7, 3, 7, 8, 3, 3, 3, 6, 0, 3, 3, 3]

In [ ]:
if run_no == 2:
    topic_task_no = round_1_topic_task_manual
else:
    topic_task_no = round_1_topic_task_automatic

In [ ]:
with open(f'RU-{round_no}-{run_no}txt', 'w') as f:
    for i in tqdm(range(30)):
        query = root[i][0].text
        task = topic_task_no[i]
        expansion = tasks[task]
        query = query.split(' ')
        for w in expansion:
            for part in w.split(' '):
                query.append(part)
        query = ' '.join(query)
        searcher = pysearch.SimpleSearcher('lucene-index-covid-full-text-2020-04-10/')
        hits = searcher.search(query, 1005)
        topicno = i + 1
        seen = set()
        j = 0
        while j < 1200:
            if hits[j].docid in seen:
                j += 1
                continue
            rank = len(seen) + 1
            hit = hits[j]
            f.write(f'{topicno} Q0 {hit.docid} {rank} {hit.score} RU-bm25-t-exp\n')
            seen |= {hits[j].docid}
            if rank == 1000:
                break
            j += 1